In [2]:
import sys

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

from sem_covid.services.dataset_pipelines.unified_dataset_pipeline import UnifiedDatasetPipeline
from sem_covid.services.model_registry import embedding_registry
from sem_covid.services.store_registry import store_registry

In [3]:
es_store = store_registry.es_index_store()
emb_model = embedding_registry.sent2vec_universal_sent_encoding()

INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [4]:
unified_dataset_pipeline = UnifiedDatasetPipeline(es_store=es_store, emb_model=emb_model)
unified_dataset_pipeline.extract()
unified_dataset_pipeline.transform()
unified_dataset_pipeline.load()
